In [39]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

In [2]:
path = os.getcwd()

# read all the files with extension .csv
filenames = glob.glob(os.path.join(path, "*.txt"))

In [113]:
#get light time inputs 
if __name__ == "__main__":
    lightsOn = input("Input lights on time (format x:xx AM/PM): ")
    lightsOff = input("Input lights off time (format x:xx AM/PM): ")

#lightsOn = '6:00 AM'
#lightsOff = '6:00 PM'
def convertTime(getTime):
    stripTime = datetime.strptime(getTime, '%I:%M %p').time()
    return stripTime
lightsOn = convertTime(lightsOn)
lightsOff = convertTime(lightsOff)

In [163]:
d_list = []
for file in filenames:

    getSubject = file.split('\\')[-1]
    getSubject = getSubject.split('.')[0]
    
    d=pd.read_csv(file, skiprows= 17, delimiter='\t',  escapechar='\\', usecols=['Time', 'Stage'])
    d=d.iloc[1: , :]
    d.Time = pd.to_datetime(d.Time)
    d['File'] = getSubject

    #strip time
    d['DateTime'] = d['Time']
    d['Time'] = d['Time'].dt.time

    d['LightsOn'] = lightsOn
    d['LightsOff'] = lightsOff

    d['IsLightsOn'] = np.where((d['Time']>=d['LightsOn']) & (d['Time']<d['LightsOff']),  1, 0)

    #time difference between time points is always four seconds
    d['StageTime'] = 4

    d['NextStage']=(d['Stage'].shift(-1))
    d['IsTransition'] = np.where(d['Stage'] == d['NextStage'], 0, 1)
    #d_list.append(d)
#DF_Stages = pd.concat(d_list, axis=0, ignore_index=True) 
   

In [164]:
lightsOnData = d['IsLightsOn'] == 1
lightsOnData = d[lightsOnData]

In [165]:
lightsOnData['StageID']=lightsOnData['IsTransition'].cumsum().astype(str)+lightsOnData['Stage']

C:\Users\andti\AppData\Local\Temp\ipykernel_37604\3997018637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lightsOnData['StageID']=lightsOnData['IsTransition'].cumsum().astype(str)+lightsOnData['Stage']


In [166]:
lightsOnData

,Time,Stage,File,DateTime,LightsOn,LightsOff,IsLightsOn,StageTime,NextStage,IsTransition,StageID
1,11:13:46,W,m013_BEEG13_Day1,2021-04-20 11:13:46,06:00:00,18:00:00,1,4,W,0,0W
2,11:13:50,W,m013_BEEG13_Day1,2021-04-20 11:13:50,06:00:00,18:00:00,1,4,W,0,0W
3,11:13:54,W,m013_BEEG13_Day1,2021-04-20 11:13:54,06:00:00,18:00:00,1,4,W,0,0W
4,11:13:58,W,m013_BEEG13_Day1,2021-04-20 11:13:58,06:00:00,18:00:00,1,4,W,0,0W
5,11:14:02,W,m013_BEEG13_Day1,2021-04-20 11:14:02,06:00:00,18:00:00,1,4,W,0,0W
...,...,...,...,...,...,...,...,...,...,...,...
21596,11:13:26,W,m013_BEEG13_Day1,2021-04-21 11:13:26,06:00:00,18:00:00,1,4,NR,1,303W
21597,11:13:30,NR,m013_BEEG13_Day1,2021-04-21 11:13:30,06:00:00,18:00:00,1,4,NR,0,303NR
21598,11:13:34,NR,m013_BEEG13_Day1,2021-04-21 11:13:34,06:00:00,18:00:00,1,4,NR,0,303NR
21599,11:13:38,NR,m013_BEEG13_Day1,2021-04-21 11:13:38,06:00:00,18:00:00,1,4,NR,0,303NR


In [169]:
lightsOnData.groupby(['StageID', "Stage"])['StageTime'].sum()

StageID  Stage
0W       W        3260
100NR    NR        124
100W     W           4
101NR    NR          4
101R     R           4
                  ... 
98W      W           4
99NR     NR          4
99W      W           4
9NR      NR        148
9W       W           4
Name: StageTime, Length: 598, dtype: int64